In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import json
states=json.load(open("states_india.geojson",'r'))
#states

In [3]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
app=dash.Dash(__name__, external_stylesheets=external_stylesheets)
d={}

df=pd.read_csv("Data.csv")



da=pd.read_csv("Rain.csv")

da=da[:4115]
for i in states['features']:
    i['id']=i['properties']['state_code']
    d[i["properties"]["st_nm"]]=i["id"]
    
df["id"]=df["place"].apply(lambda x:d[x])
da["id"] = da["State"].apply(lambda x: d[x])
da['Year']=da['Year'].astype(int)
available_indicators = da['Year'].unique()

In [ ]:
fig= px.choropleth(
      df,
      locations="id",
      geojson=states,
      color="deforestation(in Ha)",
      hover_name="place",
      scope="asia",
      color_continuous_scale='deep')
      
fig.update_geos(fitbounds="locations")
fig.update_layout(title_text='Deforestation by region')

jpg= px.choropleth(
    df,
    locations="id",
    geojson=states,
    color="rainfall(in mm)",
    hover_name='place',
    scope='asia',
    color_continuous_scale='matter')

jpg.update_geos(fitbounds="locations")
jpg.update_layout(title_text='Rainfall by region')



In [ ]:
smg=go.Figure(data=[
                go.Bar(name='rainfall', x= df['place'], y=df['rainfall(in mm)']),
                go.Bar(name='deforestation',x= df.place, y=df['deforestation(in Ha)'])
])

smg.update_layout(barmode="group")
smg.update_layout(title_text='Rainfallvs Deforestation')

In [ ]:
app.layout= html.Div(
        html.Div([
            html.Div(
                [ html.H1(children='Deforestatin and Rainfall Stats',style={'colour':'green'},)
                    ],className="row"
            ),
            
            html.Div(
                [
                html.Div([
                    dcc.Graph(
                        id='a',
                        figure=fig)
                    ],className='six columns'
                ),
                
                html.Div([
                    dcc.Graph(
                        id='b',
                        figure=jpg)
                    ], className="six columns"
                )
                ], className="row"
            ),
            
            html.Div([
                html.Div([
                    dcc.Dropdown(id="slct_year",
                         options=[
                             {'label': i, 'value': i} for i in available_indicators

                             ],
                         multi=False,
                         value=1901,
                         style={'width': "40%"}
                         ),

            html.Div(id='output_container', children=[]),
            html.Br(),

            dcc.Graph(id='my_bee_map', figure={})
            ],className="six columns"
            ),
            html.Div([
                dcc.Graph(id='the_graph',figure={}),
                dcc.RangeSlider(id='the_year',
                               min=2000,
                               max=2015,
                               value=[2000,20003],
                               step=None)
            ],className="six columns"
            )
            
            ], className="row"
            ),
            html.Div([
                dcc.Graph(id='p',figure=smg)
            ],className="row")
        ])
)

In [9]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)
    
    dff = da.copy()
    dff = dff[dff["Year"] == option_slctd]
    
    # Plotly Express
    fig = px.choropleth(
        dff,
        locations='id',
        geojson=states,
        color='Annual Rainfall',
        scope='asia',
        hover_name="State",
        hover_data=['State'],
        color_continuous_scale='matter'
    
    )
    fig.update_geos(fitbounds="locations", visible=False)

    return container, fig

In [10]:
@app.callback(
    Output('the_graph','figure'),
    [Input('the_year','value')]
)

def update_graph(years_chosen):
    dss=da[(da['Year']>=years_chosen[0] & (da['Year']<=years_chosen[1]))]
    
    scatterplot = px.scatter(
        data_frame=dss,
        x="State",
        y="Annual Rainfall",
        color="State",
        size='Year',
        hover_data=['State'],
        height=550
    )
    scatterplot.update_traces(textposition='top center')
    
    return (scatterplot)

In [ ]:
if __name__ == '__main__':
    app.run_server(use_reloader=False)